In [1]:
import json
import csv
import os
import datetime
from pprint import pprint as pr
import sqlalchemy as sa
from sqlalchemy import join
import pandas as pd
from app.sample_data import stat_vectors, comp_style_vectors, map_style_vectors, roles

In [148]:
BASE_DIR = os.path.abspath(os.path.dirname('../'))

SERVER='192.168.0.245'
SQLSERVER_DATABASE='lookingglass'
SQLSERVER_USER='thecaptain'
SQLSERVER_PASSWORD='99Redbal00ns'
PORT=1433
SQLSERVER_BASE_URI = 'mssql+pyodbc://{USER}:{PWD}@{SERVER}:{PORT}/{DATABASE}?driver=ODBC+Driver+17+for+SQL+Server'

engine = sa.create_engine(SQLSERVER_BASE_URI.format(
  SERVER=SERVER,
  DATABASE=SQLSERVER_DATABASE,
  USER=SQLSERVER_USER,
  PWD=SQLSERVER_PASSWORD,
  PORT=PORT
))

meta_data = sa.MetaData(bind=engine)
sa.MetaData.reflect(meta_data)

In [149]:
Player=meta_data.tables['Player']
TeamMember=meta_data.tables['TeamMember']
Hero=meta_data.tables['Hero']
Battleground=meta_data.tables['Map']
Replay=meta_data.tables['Replay']

In [241]:
team_id = 4109
div_id = 1024
act_list = ['TheCaptain#12345', 'Aly#11717']

# j = Replay.join(Player,Player.c.Battletag == Replay.c.Battletag) \
#     .join(Hero, Replay.c.Hero == Hero.c.HeroName)
#
# query = sa.select(
#     Replay
#     , Player.c.PlayerId
#     , Hero.c.HeroId
#     , Hero.c.NewRole
# ).select_from(j).where(sa.and_(
#      Replay.c.GameDate >= '01/01/2022'
#     , Player.c.Battletag.in_(act_list)
# ))

j = Replay.join(Player,Player.c.Battletag == Replay.c.Battletag) \
    .join(TeamMember, TeamMember.c.PlayerId == Player.c.PlayerId) \
    .join(Hero, Replay.c.Hero == Hero.c.HeroName)

query = sa.select(
    TeamMember.c.IsActive
    , Replay
    , Player.c.PlayerId
    , Hero.c.HeroId
    , Hero.c.NewRole
).select_from(j).where(sa.and_(
    TeamMember.c.TeamId == team_id
    , TeamMember.c.IsActive==True
    , Replay.c.GameDate >= '01/01/2022'
    , TeamMember.c.DivisionId==div_id
))

df = pd.read_sql(sql = query,
                 con = engine)
df['Battletag'].drop_duplicates(keep='first', inplace=True)

In [242]:
bins = [0, 7, 14, 30, 60, 90, 180]
labels = [7, 14, 30, 60, 90, 180]
df = df[df.GameDate > datetime.datetime.now() - pd.to_timedelta("180day")]
df['ReplayAgeInDays'] = ((df['GameDate'] - datetime.datetime.now()).dt.days) * -1
df['Binned'] = pd.cut(df['ReplayAgeInDays'], bins, labels=labels)
df = df[df['Binned'] <= 60]

In [243]:
points = ['Battletag', 'SourceReplayId', 'Team']
dfn = df[[*points]].groupby(['SourceReplayId', 'Team']).agg(
    Count=('SourceReplayId', 'count')
).sort_values('Count', ascending=False)

team_df = pd.merge(df, dfn[dfn['Count'] == 5], right_on=['SourceReplayId', 'Team'], left_on=['SourceReplayId', 'Team'], how='inner')

In [244]:
points = ['Battletag', 'Binned', 'GameType', 'IsWinner']
dfn = team_df[[*points]].groupby(['Battletag', 'GameType', 'Binned']).agg(
    Count=('GameType', 'count')
    , Wins=('IsWinner', 'sum')
).sort_values('Count', ascending=False)

dfn['WinRate'] = round(dfn.Wins / dfn.Count, 2)
dfn = dfn.groupby(['Battletag']).head(100).sort_index().reset_index()

pd.crosstab(index=[dfn.Binned], columns=[dfn.GameType], values=dfn.Count, aggfunc='sum', margins=True)

GameType,Custom,Quick Match,Storm League,All
Binned,,,,
7,0,5,0,5
14,5,10,30,45
30,45,20,45,110
60,30,15,50,95
90,0,0,0,0
180,0,0,0,0
All,80,50,125,255


In [239]:
points = ['Battletag', 'Binned', 'Hero', 'IsWinner']
dfn = df[[*points]].groupby(['Battletag', 'Hero', 'Binned']).agg(
    Count=('Hero', 'count')
    , Wins=('IsWinner', 'sum')
).sort_values('Count', ascending=False)

dfn['WinRate'] = round(dfn.Wins / dfn.Count, 2)
dfn.reset_index(inplace=True)
dfn = dfn[dfn['Binned'] <= 60].groupby(['Hero']).head(100).sort_index()
dfn.set_index(['Hero', 'Binned', 'Battletag' ], inplace=True)
dfn.head(100)

,,,Count,Wins,WinRate
Hero,Binned,Battletag,,,
Jaina,30,TheCaptain#12345,8,4,0.50
Stukov,60,BornToShine#11556,8,3,0.38
Johanna,30,MowBWan#1362,7,3,0.43
Anduin,30,BornToShine#11556,6,4,0.67
Stukov,30,BornToShine#11556,6,3,0.50
...,...,...,...,...,...
Diablo,60,Knivers#1675,1,1,1.00
Genji,14,SubZero#1577,1,0,0.00
Lúcio,14,BornToShine#11556,1,0,0.00


In [240]:
role_points = ['Battletag', 'NewRole', 'IsWinner', 'Binned']
dfn = df[[*role_points]].groupby(['Battletag', 'NewRole', 'Binned']).agg(
    Count=('NewRole', 'count')
    , Wins=('IsWinner', 'sum')
).sort_values('Count', ascending=False)

dfn['WinRate'] = round(dfn.Wins / dfn.Count, 2)
dfn.reset_index(inplace=True)
dfn = dfn[dfn['Binned'] <= 60].groupby(['NewRole']).head(100).sort_index()
dfn.set_index(['NewRole', 'Binned', 'Battletag' ], inplace=True)
dfn.head(100)

Count  Wins  WinRate
NewRole         Binned Battletag                              
Healer          30     BornToShine#11556     21    12     0.57
Tank            30     MowBWan#1362          19    12     0.63
Bruiser         30     DirePants#125315      19    12     0.63
Healer          60     BornToShine#11556     19    10     0.53
Ranged Assassin 30     TheCaptain#12345      15     8     0.53
...                                         ...   ...      ...
Support         30     SubZero#1577           0     0      NaN
                60     SubZero#1577           0     0      NaN
Tank            7      SubZero#1577           0     0      NaN
                14     SubZero#1577           0     0      NaN
Ranged Assassin 60     YipYip#11141           0     0      NaN

[100 rows x 3 columns]

In [219]:
dfn = df.groupby(['Hero']).agg(
    Count=('NewRole', 'count')
    , Wins=('IsWinner', 'sum')
    , MeanLevel=('Level', 'mean')
    , MeanDeaths=('Deaths', 'mean')
    , MeanKills=('Kills', 'mean')
    , MeanAssists=('Assists', 'mean')
)

dfn['WinRate'] = round(dfn.Wins / dfn.Count, 2)
dfn = dfn.groupby(['Hero']).head(5).sort_index()
dfn.head(100)

,Count,Wins,MeanLevel,MeanDeaths,MeanKills,MeanAssists,WinRate
Hero,,,,,,,
Anub'arak,11,7,20.363636,3.363636,1.636364,8.363636,0.64
Auriel,1,0,19.000000,5.000000,0.000000,5.000000,0.00
Blaze,1,0,19.000000,6.000000,1.000000,3.000000,0.00
Brightwing,7,4,20.000000,2.285714,2.142857,11.428571,0.57
Cassia,90,56,20.200000,2.211111,4.233333,8.711111,0.62
Chromie,9,3,19.888889,1.777778,3.222222,7.333333,0.33
Deckard,3,1,17.666667,2.000000,0.333333,12.666667,0.33
Dehaka,3,0,20.666667,4.666667,0.333333,7.333333,0.00
Diablo,4,2,23.250000,7.500000,2.000000,11.000000,0.50


In [126]:
bins = [0, 30, 60, 90]
labels =[30,60,90]
dfn = df[df.GameDate > datetime.datetime.now() - pd.to_timedelta("90day")][['SourceReplayId', 'GameDate', ]]
dfn['ReplayAgeInDays'] = ((dfn['GameDate'] - datetime.datetime.now()).dt.days) * -1
dfn['Binned'] = pd.cut(dfn['ReplayAgeInDays'], bins,labels=labels)
dfn.groupby('Binned').agg(Count=('Binned', 'count'))

,Count
Binned,
30,320
60,310
90,242
